# 🏥 Visual AI in Healthcare with FiftyOne - MedSAM2 on CT Scans  
**Empowering medical imaging workflows with open-source tools and modern AI**

This notebook is part of the **“Visual AI in Healthcare with FiftyOne”** workshop. Here, we explore how to **load and prepare CT video data with MedSAM2 annotations** from Hugging Face into FiftyOne for visual AI workflows in healthcare.

🔬 **What you’ll learn in this notebook:**

- How to **load the BTCV-CT-as-video-MedSAM2 dataset** from Hugging Face using FiftyOne utilities  
- How to **preview CT scans as video sequences** within the FiftyOne App  
- How to **inspect and filter frame-level detections** to isolate relevant slices   

📚 **Part of the notebook series:**
1. `01_load_arcade_dataset.ipynb` – Load and visualize the ARCADE dataset.  
2. `02_load_deeplesion_balanced.ipynb` – Curate and balance the DeepLesion dataset.  
3. `03_vlms_analysis_arcade.ipynb` – Use VFMs like NVLabs_CRADIOV3 in dataset undersatnding for ARCADE. 
4. `04_finetune_yolo8_stenosis.ipynb` – Train and integrate YOLOv8 for stenosis detection.  
5. `05_medsam2_ct_scan.ipynb` – Run MedSAM2 on CT scans for segmentation.  
6. `06_nvidia_vista_segmentation.ipynb` – Explore NVIDIA-VISTA-3D.  
7. `07_medgemma_vqa.ipynb` – Perform visual question answering and classification with MedGemma.

All notebooks are standalone but are best experienced sequentially.


### ✅ Requirements

Please install all the requeriments for running this notebook

In [ ]:
#!pip install datasets fiftyone pandas

Clean up previous dataset (if needed)

Before loading the dataset, we ensure there's no existing dataset with the same name in the FiftyOne environment.

- The dataset we're working with is the **BTCV-CT-as-video-MedSAM2**, hosted on Hugging Face and loaded via FiftyOne.
- FiftyOne stores datasets locally, so we check if one with the same name exists and delete it to avoid conflicts.


In [ ]:
import fiftyone as fo

# Name used internally by FiftyOne (it does not use slashes like 'Voxel51/BTCV-...')
dataset_name = "Voxel51/BTCV-CT-as-video-MedSAM2-dataset"

# Delete the dataset if it exists
if fo.dataset_exists(dataset_name):
    fo.delete_dataset(dataset_name)
    print(f"Deleted existing dataset: {dataset_name}")
else:
    print(f"No dataset found with name: {dataset_name}")


### Load BTCV CT Scan Video Dataset from Hugging Face

In this step, we load a sample from the **BTCV-CT-as-video-MedSAM2** dataset using FiftyOne’s integration with Hugging Face.

- We import the dataset using `load_from_hub()` and keep only the first 2 samples to simplify our analysis.
- This dataset is structured as **videos of CT scans** with **MedSAM2-predicted segmentations**.
- We then **filter out all frames except the middle one (frame 100)** to focus on a single annotated frame for each video.
- This is necessary because MedSAM2 only propagates masks forward in video, and this step helps us simplify the visualization.


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
from fiftyone.utils.huggingface import load_from_hub


dataset = load_from_hub("Voxel51/BTCV-CT-as-video-MedSAM2-dataset")[:2]


# Retaining detections from a single frame in the middle
# Note that SAM2 only propagates segmentation masks forward in a video
(
    dataset
    .match_frames(F("frame_number") != 100)
    .set_field("frames.gt_detections", None)
    .save()
)

### 🛠️ Install and Validate `sam2` Package

To run MedSAM2 locally or inspect its behavior, we clone and install the official [facebookresearch/sam2](https://github.com/facebookresearch/sam2) repository.

However, an issue may arise during installation due to the version of `setuptools`. If you encounter a build error, modify the `pyproject.toml` file as follows:

**Fix for `setuptools` version issue:**

In the file `sam2/pyproject.toml`, update the dependency line:

```toml
# Before:
setuptools>=61.0

# After:
setuptools>=62.3.0,<75.9


In [ ]:
#git clone https://github.com/facebookresearch/sam2.git && cd sam2
#pip install -e .

import sam2
print(hasattr(sam2, 'build_sam'))


### 🤖 Apply MedSAM2 Video Segmentation Model

We now load the **MedSAM2 video model** from the FiftyOne Model Zoo and apply it to our dataset. This model supports segmenting anatomical structures across frames in a medical video.

- **Model**: `med-sam-2-video-torch`  
- **Prompt**: Uses `frames.gt_detections` (bounding boxes) as the prompt to generate masks  
- **Output**: Stores results in `frames.pred_segmentations`


In [ ]:
model = foz.load_zoo_model("med-sam-2-video-torch")

# Segment inside boxes and propagate to all frames
dataset.apply_model(
    model,
    label_field="pred_segmentations",
    prompt_field="frames.gt_detections",
)

session = fo.launch_app(dataset)